In [1]:
from transVAE.model.transVAE import transVAE
import gc

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [26]:
import scanpy as sc
adata = sc.read_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/train_adata_baseline.h5ad")

In [29]:
import pandas as pd

In [30]:
genes_intersection = pd.read_table("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/make_big_dataset/genes_shared_across_datasets.txt")

In [31]:
genes_intersection = genes_intersection.genes_shared_across_datasets.tolist()

In [33]:
adata = adata[:,adata.var.index.isin(genes_intersection)]

In [35]:
adata_train = transVAE.setup_anndata(adata, categorical_covariate_embed_keys=["dataset", "cell_type"], categorical_covariate_keys=["organism"], copy = True)

INFO     Using column names from columns of adata.obsm['covariates']                                               
INFO     Using column names from columns of adata.obsm['covariates_embed']                                         


In [36]:
dataset_to_predict = "wang_to_predict_cleanCT.h5ad"
dataset_ground_truth1 = "dbdb_ground_truth_cleanCT.h5ad"
dataset_ground_truth2 = "mSTZ_ground_truth_cleanCT.h5ad"

In [38]:
    # reading val data and data to translate
    adata_to_predict = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/data_to_predict/{dataset_to_predict}")
    dbdb_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth1}")
    mSTZ_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth2}")

Must include keys 'adata_to_predict','translate_dicts', and 'ground_truths' for operations within the method.

In [46]:
adata_to_predict = adata_to_predict[:,adata_to_predict.var.index.isin(genes_intersection)]
adata_to_predict = adata_to_predict[:,adata_to_predict.var.sort_index().index]
dbdb_ground_truth = dbdb_ground_truth[:,dbdb_ground_truth.var.index.isin(genes_intersection)]
mSTZ_ground_truth = mSTZ_ground_truth[:,mSTZ_ground_truth.var.index.isin(genes_intersection)]

In [57]:
adata_to_predict_h = adata_to_predict[adata_to_predict.obs.disease == "healthy"]
adata_to_predict_OOD = adata_to_predict[adata_to_predict.obs.disease == "T2D"]

dbdb_ground_truth_h = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "healthy"]
dbdb_ground_truth_OOD = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "T2D"]

mSTZ_ground_truth_h = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "healthy"]
mSTZ_ground_truth_OOD = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "T2D"]

In [48]:
validation_adatas_dict = {"adata_to_predict":adata_to_predict_OOD,
                          "ground_truths": {"dbdb": dbdb_ground_truth_OOD,
                                            "mSTZ": mSTZ_ground_truth_OOD},
                          "translate_dicts": {"dbdb":{"dataset":"db/db", "organism": "Mus musculus"},
                                              "mSTZ":{"dataset":"mSTZ", "organism": "Mus musculus"}}}

In [49]:
# make the model
model = transVAE(adata_train, 
                 n_hidden=3000,
                 n_latent=512,
                 n_layers=6, 
                 dropout_rate=0.3, 
                 cov_embed_dims = 10, 
                 kl_weight = 0.0005,
                 validation_adatas_dict = validation_adatas_dict)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [50]:
# train the model
model.train(batch_size=4096, max_epochs = 10, train_size = 0.99, enable_progress_bar = True,
            early_stopping = True, early_stopping_monitor = 'reconstruction_loss_validation', early_stopping_min_delta = 0.1, early_stopping_patience = 100,
            plan_kwargs = {"lr": 0.00001,
                           "weight_decay":0.000001,
                           "reduce_lr_on_plateau":True,
                           "lr_factor":0.5,
                           "lr_patience":50,
                           "lr_scheduler_metric":"reconstruction_loss_validation"})

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
SLURM auto-requeueing enabled. Setting signal handlers.


Epoch 6/10:  50%|█████     | 5/10 [02:40<02:40, 32.02s/it, v_num=1, train_loss_step=1.21e+3, train_loss_epoch=1.3e+3] 

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [11]:
from transVAE.module._utils import check_adatas_var_index, compare_de, compare_logfold,compute_r2_score

In [59]:
results = model.history

# dbdb
preds_dbdb_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})
preds_dbdb_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})

# mSTZ
preds_mSTZ_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})
preds_mSTZ_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [17]:
# dbdb
results["r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h, dbdb_ground_truth_h)
results["r2_dbdb_OOD"] = model.history["dbdb_r2_validation_eval"].iloc[-1].item()
# mSTZ
results["r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h, mSTZ_ground_truth_h)
results["r2_mSTZ_OOD"] = model.history["mSTZ_r2_validation_eval"].iloc[-1].item()

In [18]:
results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD.X.toarray(), Y=preds_dbdb_OOD.X, C=dbdb_ground_truth_h.X.toarray(), shared_top = 1000)
results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD.X.toarray(), Y=preds_mSTZ_OOD.X, C=mSTZ_ground_truth_h.X.toarray(), shared_top = 1000)

for name, value in results_de_dbdb.items():
    results[name + "_dbdb"] = value
for name, value in results_de_mSTZ.items():
    results[name + "_mSTZ"] = value

In [130]:
for ct in dbdb_ground_truth.obs.cell_type.unique():
    # healthy splits
    preds_dbdb_h_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
    dbdb_ground_truth_h_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
    preds_mSTZ_h_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
    mSTZ_ground_truth_h_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]

    # OOD splits
    preds_dbdb_OOD_tmp = preds_dbdb_OOD[preds_dbdb_OOD.obs.cell_type == ct]
    dbdb_ground_truth_OOD_tmp = dbdb_ground_truth_OOD[dbdb_ground_truth_OOD.obs.cell_type == ct]
    preds_mSTZ_OOD_tmp = preds_mSTZ_OOD[preds_mSTZ_OOD.obs.cell_type == ct]
    mSTZ_ground_truth_OOD_tmp = mSTZ_ground_truth_OOD[mSTZ_ground_truth_OOD.obs.cell_type == ct]

    ct = ct.replace(" ", "_")

    results[ct + "_" + "r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h_tmp, dbdb_ground_truth_h_tmp)
    results[ct + "_" + "r2_dbdb_OOD"] = compute_r2_score(preds_mSTZ_OOD_tmp, mSTZ_ground_truth_OOD_tmp)
    # mSTZ
    results[ct + "_" + "r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h_tmp, mSTZ_ground_truth_h_tmp)
    results[ct + "_" + "r2_mSTZ_OOD"] = compute_r2_score(preds_mSTZ_OOD_tmp, mSTZ_ground_truth_OOD_tmp)

    results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD_tmp.X.toarray(), Y=preds_dbdb_OOD_tmp.X, C=dbdb_ground_truth_h_tmp.X.toarray(), shared_top = 4000, method = "t-test")
    results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD_tmp.X.toarray(), Y=preds_mSTZ_OOD_tmp.X, C=mSTZ_ground_truth_h_tmp.X.toarray(), shared_top = 4000, method = "t-test")

    for name, value in results_de_dbdb.items():
        results[ct + "_" + name + "_dbdb" ] = value
    for name, value in results_de_mSTZ.items():
        results[ct + "_" + name + "_mSTZ_"] = value


In [69]:
ct = ct.replace("_", " ")

In [75]:
de_results = compare_de(X=dbdb_ground_truth_OOD_tmp.X.toarray(), Y=preds_dbdb_OOD_tmp.X, C=dbdb_ground_truth_h_tmp.X.toarray(), shared_top = 4000, method = "t-test")

In [77]:
de_results

{'shared_top_genes': 0.28025,
 'scores_corr': nan,
 'pvals_adj_corr': nan,
 'scores_ranks_corr': -0.04854936040169768}

In [123]:
dbdb_ground_truth_OOD_tmp

View of AnnData object with n_obs × n_vars = 1373 × 13237
    obs: 'disease', 'dataset', 'organism', 'cell_type'

In [124]:
dbdb_ground_truth_h_tmp

View of AnnData object with n_obs × n_vars = 1373 × 13237
    obs: 'disease', 'dataset', 'organism', 'cell_type'

In [90]:
import numpy as np
import anndata as ad
import pandas as pd
import scanpy as sc

X=dbdb_ground_truth_OOD_tmp.X.toarray()
Y=preds_dbdb_OOD_tmp.X
C=dbdb_ground_truth_h_tmp.X.toarray()
shared_top = 4000

"""Compare DEG across real and simulated perturbations.

Computes DEG for real and simulated perturbations vs. control and calculates
metrics to evaluate similarity of the results.

Args:
    X: Real perturbed data.
    Y: Simulated perturbed data.
    C: Control data
    shared_top: The number of top DEG to compute the proportion of their intersection.
    **kwargs: arguments for `scanpy.tl.rank_genes_groups`.
"""

n_vars = X.shape[1]
assert n_vars == Y.shape[1] == C.shape[1]

shared_top = min(shared_top, n_vars)
vars_ranks = np.arange(1, n_vars + 1)

adatas_xy = {}
adatas_xy["x"] = ad.AnnData(X, obs={"label": "comp"})
adatas_xy["y"] = ad.AnnData(Y, obs={"label": "comp"})
adata_c = ad.AnnData(C, obs={"label": "ctrl"})

results = pd.DataFrame(index=adata_c.var_names)
top_names = []
for group in ("x", "y"):
    adata_joint = ad.concat((adatas_xy[group], adata_c), index_unique="-")

    sc.tl.rank_genes_groups(adata_joint, groupby="label", reference="ctrl", key_added="de",method = "t-test")

    srt_idx = np.argsort(adata_joint.uns["de"]["names"]["comp"])
    results[f"scores_{group}"] = adata_joint.uns["de"]["scores"]["comp"][srt_idx]
    results[f"pvals_adj_{group}"] = adata_joint.uns["de"]["pvals_adj"]["comp"][srt_idx]
    # needed to avoid checking rankby_abs
    results[f"ranks_{group}"] = vars_ranks[srt_idx]

    top_names.append(adata_joint.uns["de"]["names"]["comp"][:shared_top])

metrics = {}
metrics["shared_top_genes"] = len(set(top_names[0]).intersection(top_names[1])) / shared_top
metrics["scores_corr"] = results["scores_x"].corr(results["scores_y"], method="pearson")
metrics["pvals_adj_corr"] = results["pvals_adj_x"].corr(results["pvals_adj_y"], method="pearson")
metrics["scores_ranks_corr"] = results["ranks_x"].corr(results["ranks_y"], method="spearman")

In [114]:
adata_joint = ad.concat((adatas_xy["x"], adata_c), index_unique="-")

In [115]:
sc.tl.rank_genes_groups(adata_joint, groupby="label", reference="ctrl", key_added="de",method = "t-test")

srt_idx = np.argsort(adata_joint.uns["de"]["names"]["comp"])
results[f"scores_{group}"] = adata_joint.uns["de"]["scores"]["comp"][srt_idx]
results[f"pvals_adj_{group}"] = adata_joint.uns["de"]["pvals_adj"]["comp"][srt_idx]
# needed to avoid checking rankby_abs
results[f"ranks_{group}"] = vars_ranks[srt_idx]

top_names.append(adata_joint.uns["de"]["names"]["comp"][:shared_top])

In [118]:
adata_joint.uns["de"]

{'params': {'groupby': 'label',
  'reference': 'ctrl',
  'method': 't-test',
  'use_raw': False,
  'layer': None,
  'corr_method': 'benjamini-hochberg'},
 'names': rec.array([('13236',), ('4407',), ('4417',), ..., ('8819',), ('8818',),
            ('0',)],
           dtype=[('comp', 'O')]),
 'scores': rec.array([(0.,), (0.,), (0.,), ..., (0.,), (0.,), (0.,)],
           dtype=[('comp', '<f4')]),
 'pvals': rec.array([(1.,), (1.,), (1.,), ..., (1.,), (1.,), (1.,)],
           dtype=[('comp', '<f8')]),
 'pvals_adj': rec.array([(1.,), (1.,), (1.,), ..., (1.,), (1.,), (1.,)],
           dtype=[('comp', '<f8')]),
 'logfoldchanges': rec.array([(0.,), (0.,), (0.,), ..., (0.,), (0.,), (0.,)],
           dtype=[('comp', '<f4')])}

In [104]:
results["scores_x"].corr(results["scores_y"], method="pearson")

nan

In [94]:
results["scores_x"]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
13232    0.0
13233    0.0
13234    0.0
13235    0.0
13236    0.0
Name: scores_x, Length: 13237, dtype: float32

In [95]:
results["scores_y"]

0          12.257668
1         187.306656
2        1019.721008
3        1078.150513
4          13.917542
            ...     
13232     797.833862
13233     157.559326
13234      41.369530
13235     806.106323
13236      41.402588
Name: scores_y, Length: 13237, dtype: float32

In [93]:
results["scores_x"].corr(results["scores_y"], method="pearson")

nan

In [84]:
sc.tl.rank_genes_groups(adata_joint, groupby="label", reference="ctrl", key_added="de",method = "t-test")

In [88]:
adata_joint.uns["de"]

{'params': {'groupby': 'label',
  'reference': 'ctrl',
  'method': 't-test',
  'use_raw': False,
  'layer': None,
  'corr_method': 'benjamini-hochberg'},
 'names': rec.array([('1952',), ('11927',), ('13115',), ..., ('12282',), ('12281',),
            ('12285',)],
           dtype=[('comp', 'O')]),
 'scores': rec.array([(1233.5737 ,), (1221.8506 ,), (1221.5106 ,), ...,
            (-297.07452,), (-311.3268 ,), (-330.33182,)],
           dtype=[('comp', '<f4')]),
 'pvals': rec.array([(0.,), (0.,), (0.,), ..., (0.,), (0.,), (0.,)],
           dtype=[('comp', '<f8')]),
 'pvals_adj': rec.array([(0.,), (0.,), (0.,), ..., (0.,), (0.,), (0.,)],
           dtype=[('comp', '<f8')]),
 'logfoldchanges': rec.array([(29.131117 ,), (29.102427 ,), (29.198795 ,), ...,
            (-4.4561553,), (-4.488192 ,), (-4.8284407,)],
           dtype=[('comp', '<f4')])}

In [70]:
preds_dbdb_h.obs.cell_type

A0019_AAACAGCCAGGACCAA-1    type B pancreatic cell
A0019_AAACAGCCATGAGTTT-1    type B pancreatic cell
A0019_AAACATGCAGCTTAAT-1         pancreatic A cell
A0019_AAACCAACACTATGGC-1    type B pancreatic cell
A0019_AAACGCGCAGCTAACC-1         pancreatic A cell
                                     ...          
C0027_TTTGTTGGTGAGGTAG-1         pancreatic A cell
C0027_TTTGTTGGTGCTCCGT-1    type B pancreatic cell
C0027_TTTGTTGGTGTGTCCC-1    type B pancreatic cell
C0027_TTTGTTGGTTTCGCGC-1    type B pancreatic cell
C0027_TTTGTTGGTTTGACCT-1    type B pancreatic cell
Name: cell_type, Length: 18274, dtype: category
Categories (3, object): ['pancreatic A cell', 'type B pancreatic cell', 'pancreatic D cell']

In [71]:
preds_dbdb_h_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
dbdb_ground_truth_h_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
preds_mSTZ_h_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
mSTZ_ground_truth_h_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]

# OOD splits
preds_dbdb_OOD_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
dbdb_ground_truth_OOD_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
preds_mSTZ_OOD_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
mSTZ_ground_truth_OOD_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]

ct = ct.replace(" ", "_")

In [18]:
import numpy as np
path = "/d/hpc/projects/FRI/DL/mo6643/MSC/cross_species_prediction_save/dbdb_ground_truth_OOD.npy"
np.save(path, dbdb_ground_truth_OOD.X.toarray())

In [2]:
import logging
from sacred import Experiment
import seml
import os
from transVAE.module._utils import check_adatas_var_index, compare_de, compare_logfold, compute_r2_score
import scanpy as sc
import scvi
import numpy as np
import gc
import pandas as pd

In [ ]:
extras = ['train_adata_baseline.h5ad','extra_mouse_Embryonic.h5ad','extra_human_chem.h5ad','embedding_top100_mouse.h5ad','random_mouse_seed_42.h5ad',
          'embedding_top70_human.h5ad', 'extra_human_preT2D.h5ad','random_human_seed_42.h5ad','extra_human_neonatal.h5ad','extra_mouse_chem.h5ad',
          'extra_mouse_T1D.h5ad','extra_mouse_young.h5ad']

genes_intersection = pd.read_table("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/make_big_dataset/genes_shared_across_datasets.txt")
genes_intersection = genes_intersection.genes_shared_across_datasets.tolist()

adatas = []
for extra in extras:
    adata = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/{extra}")
    adata = adata[:,adata.var.index.isin(genes_intersection)]
    adatas.apped(adata)

adata_train = sc.concat(adatas, join = "inner", axis = 0)
adata_train = transVAE.setup_anndata(adata_train, categorical_covariate_embed_keys=["dataset"], categorical_covariate_keys=["organism"], copy = True)

# reading val data and data to translate
adata_to_predict = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/data_to_predict/{dataset_to_predict}")
dbdb_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth1}")
mSTZ_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth2}")

adata_to_predict = adata_to_predict[:,adata_to_predict.var.index.isin(genes_intersection)]
adata_to_predict = adata_to_predict[:,adata_to_predict.var.sort_index().index]
dbdb_ground_truth = dbdb_ground_truth[:,dbdb_ground_truth.var.index.isin(genes_intersection)]
mSTZ_ground_truth = mSTZ_ground_truth[:,mSTZ_ground_truth.var.index.isin(genes_intersection)]

scvi.settings.seed = seed

logging.info(f'n_hidden: {n_hidden}, n_latent: {n_latent}, learning_rate: {learning_rate}, cov_embed_dims: {cov_embed_dims}, weight_decay {weight_decay}')
logging.info(f'max_epochs: {max_epochs}, n_layers: {n_layers}, dropout_rate: {dropout_rate}, kl_weight: {kl_weight}')

check_adatas_var_index(adata_train, adata_to_predict, dbdb_ground_truth, mSTZ_ground_truth)

# check this data (full_datasets)

adata_to_predict_h = adata_to_predict[adata_to_predict.obs.disease == "healthy"]
adata_to_predict_OOD = adata_to_predict[adata_to_predict.obs.disease == "T2D"]
del adata_to_predict
gc.collect()

dbdb_ground_truth_h = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "healthy"]
dbdb_ground_truth_OOD = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "T2D"]
del dbdb_ground_truth
gc.collect()

mSTZ_ground_truth_h = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "healthy"]
mSTZ_ground_truth_OOD = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "T2D"]
del mSTZ_ground_truth
gc.collect()

validation_adatas_dict = {"adata_to_predict":adata_to_predict_OOD,
                          "ground_truths": {"dbdb": dbdb_ground_truth_OOD,
                                            "mSTZ": mSTZ_ground_truth_OOD},
                          "translate_dicts": {"dbdb":{"dataset":"db/db", "organism": "Mus musculus"},
                                              "mSTZ":{"dataset":"mSTZ", "organism": "Mus musculus"}}}


# make the model
model = transVAE(adata_train, 
                 n_hidden=n_hidden, 
                 n_latent=n_latent, 
                 n_layers=n_layers, 
                 dropout_rate=dropout_rate, 
                 cov_embed_dims = cov_embed_dims, 
                 kl_weight = kl_weight,
                 validation_adatas_dict = validation_adatas_dict)

# train the model
model.train(batch_size=4096, max_epochs = max_epochs, train_size = 0.99, enable_progress_bar = True,
            early_stopping = True, early_stopping_monitor = 'mean_r2_validation_eval', early_stopping_mode = "max", 
            early_stopping_min_delta = 0.01, early_stopping_patience = 70,
            plan_kwargs = {"lr":learning_rate,
                           "weight_decay":weight_decay,
                           "reduce_lr_on_plateau":True,
                           "lr_factor":0.5,
                           "lr_patience":50,
                           "lr_scheduler_metric":"reconstruction_loss_validation"})

results = model.history

# dbdb
preds_dbdb_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})
preds_dbdb_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})

# mSTZ
preds_mSTZ_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})
preds_mSTZ_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})

# dbdb
results["r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h, dbdb_ground_truth_h)
results["r2_dbdb_OOD"] = model.history["dbdb_r2_validation_eval"].iloc[-1].item()
# mSTZ
results["r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h, mSTZ_ground_truth_h)
results["r2_mSTZ_OOD"] = model.history["mSTZ_r2_validation_eval"].iloc[-1].item()

results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD.X.toarray(), Y=preds_dbdb_OOD.X, C=dbdb_ground_truth_h.X.toarray(), shared_top = 1000, method = "t-test")
results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD.X.toarray(), Y=preds_mSTZ_OOD.X, C=mSTZ_ground_truth_h.X.toarray(), shared_top = 1000, method = "t-test")

for name, value in results_de_dbdb.items():
    results[name + "_dbdb"] = value
for name, value in results_de_mSTZ.items():
    results[name + "_mSTZ"] = value

for ct in dbdb_ground_truth_OOD.obs.cell_type.unique():

    # healthy splits
    preds_dbdb_h_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
    dbdb_ground_truth_h_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
    preds_mSTZ_h_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
    mSTZ_ground_truth_h_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]

    # OOD splits
    preds_dbdb_OOD_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
    dbdb_ground_truth_OOD_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
    preds_mSTZ_OOD_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
    mSTZ_ground_truth_OOD_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]

    ct = ct.replace(" ", "_")

    results[ct + "_" + "r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h_tmp, dbdb_ground_truth_h_tmp)
    results[ct + "_" + "r2_dbdb_OOD"] = compute_r2_score(preds_mSTZ_OOD_tmp, mSTZ_ground_truth_OOD_tmp)
    # mSTZ
    results[ct + "_" + "r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h_tmp, mSTZ_ground_truth_h_tmp)
    results[ct + "_" + "r2_mSTZ_OOD"] = compute_r2_score(preds_mSTZ_OOD_tmp, mSTZ_ground_truth_OOD_tmp)

    results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD_tmp.X.toarray(), Y=preds_dbdb_OOD_tmp.X, C=dbdb_ground_truth_h_tmp.X.toarray(), shared_top = 1000, method = "t-test")
    results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD_tmp.X.toarray(), Y=preds_mSTZ_OOD_tmp.X, C=mSTZ_ground_truth_h_tmp.X.toarray(), shared_top = 1000, method = "t-test")

    for name, value in results_de_dbdb.items():
        results[ct + "_" + name + "_dbdb" ] = value
    for name, value in results_de_mSTZ.items():
        results[ct + "_" + name + "_mSTZ_"] = value

if save_models:
    try:
        save_dir = f"/d/hpc/projects/FRI/DL/mo6643/MSC/cross_species_prediction_save/{save_folder_name}"
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_dir = os.path.join(save_dir, f"transVAE_train_{addl_dataset_train}_hid{n_hidden}_lat{n_latent}_lr{learning_rate}_cov{cov_embed_dims}_ep{max_epochs}_ly{n_layers}_dr{dropout_rate}_kl{kl_weight}_wd{weight_decay}_s{seed}")
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        #np.save(arr=preds_dbdb_h.X, file=os.path.join(save_dir,"preds_dbdb_h.npy"))
        np.save(arr=preds_dbdb_OOD.X, file=os.path.join(save_dir,"preds_dbdb_OOD.npy"))
        #np.save(arr=preds_mSTZ_h.X, file=os.path.join(save_dir,"preds_mSTZ_h.npy"))
        np.save(arr=preds_mSTZ_OOD.X, file=os.path.join(save_dir,"preds_mSTZ_OOD.npy"))
    except:
        print("An error accured while saving model predictions")

return results